In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import pandas as pd
import numpy as np
import tensorflow as tf


**Load Normal dataset**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
main_directory  = '/content/drive/MyDrive/Ass_1/train_Normal_128.npy'
main_directory1  = '/content/drive/MyDrive/Ass_1/test_Normal_128.npy'
train_Normal = np.load(main_directory)
test_Normal = np.load(main_directory1)
print(train_Normal)
print(test_Normal)

[[[[ 53.]
   [ 59.]
   [ 67.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  [[ 50.]
   [ 59.]
   [ 70.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  [[ 47.]
   [ 57.]
   [ 68.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  ...

  [[  0.]
   [  0.]
   [  0.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  [[  0.]
   [  0.]
   [  0.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  [[  0.]
   [  0.]
   [  0.]
   ...
   [  0.]
   [  0.]
   [  0.]]]


 [[[  0.]
   [  0.]
   [  0.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  [[  0.]
   [  0.]
   [  0.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  [[  0.]
   [  0.]
   [  0.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  ...

  [[  0.]
   [  0.]
   [  0.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  [[  0.]
   [  0.]
   [  0.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  [[  0.]
   [  0.]
   [  0.]
   ...
   [  0.]
   [  0.]
   [  0.]]]


 [[[  4.]
   [ 22.]
   [ 34.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  [[  4.]
   [ 18.]
   [ 33.]
   ...
   [  0.]
   [  0.]
   [  0.]]

  [[  1.]
   [ 1

In [5]:
from torch import tensor
traindata = [train_Normal[i] for i in range(len(train_Normal))]
#train = torch.stack([d[0] for d in traindata], dim=0)
train =torch.from_numpy(np.array(traindata))
train=train[0:59999]        
ys = [d[1] for d in traindata]
train_y = tensor(ys)

testdata = [test_Normal[i] for i in range(len(test_Normal))]
#test = torch.stack([d[0] for d in testdata], dim=0)  
#test =torch.from_numpy(np.array(testdata))     
Ntest=np.vstack(testdata).astype(np.float64)
test =torch.from_numpy(Ntest)
test=test[0:9999]
ys = [d[1] for d in testdata]
Ny=np.vstack(ys).astype(np.float64)
test_y = tensor(Ny)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  import sys


In [6]:

x=int(torch.rand(1,)*1000)
train = torch.rand(6000, 128, 128, 1)          
test = torch.rand(6000, 128, 128, 1) 
train_y = torch.randint(83,x, (1000,)) 
print(train_y)
test_y = torch.randint(83,x, (1000,))
print(test_y)

tensor([758, 267, 762, 243, 845, 616, 788, 589, 682, 453, 831, 355, 486, 671,
        616, 290, 491, 396, 350, 686, 676, 660, 221, 177, 108, 509, 788, 821,
        146, 691, 371, 736, 245, 632, 580, 546, 106, 281, 183, 258, 684, 659,
        120, 510, 382, 796, 161, 741, 224, 579, 793, 220, 613, 552, 882, 841,
        818, 136, 268, 412, 618, 541, 620, 772, 894, 188, 455, 653, 459, 551,
        883, 442, 146, 426, 541, 533, 282, 309, 826, 326, 646, 397, 231,  85,
        406, 385, 333, 226, 681, 323, 174, 425, 639, 196, 209, 101, 841, 666,
        880, 778, 788, 805, 336, 883, 739, 857, 716, 761, 556, 505, 716, 843,
        310, 716, 434, 415, 138, 376, 410, 854, 514, 498, 149, 853, 822, 786,
        107, 558, 114, 792, 200, 840, 750, 904, 684, 470, 303, 164, 891, 332,
        853, 101, 220, 378, 728, 813, 534, 638, 524, 749, 696, 334, 347, 119,
        606, 758, 121, 201, 733, 114, 583, 442, 489, 370, 385, 267, 739, 579,
        753, 518, 724, 487,  97, 838, 380, 330, 819, 383, 675, 1

**CNN**

*Model Architeture*

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.cnn_layer_1 = nn.Conv2d(in_channels=1, out_channels=16,kernel_size=5, stride=1, padding=2)
        self.cnn_layer_2 = nn.Conv2d(in_channels=16, out_channels=32,kernel_size=5, stride=1, padding=2)
        
        self.flatten = nn.Flatten()
        self.maxpool = nn.MaxPool2d(2,2)
        
        self.linear_layer_1 = nn.Linear(32*32*32, 900) 
        self.linear_layer_2 = nn.Linear(900, 768)
        self.linear_layer_3 = nn.Linear(768, 512)   
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(.2)
        
        # self.flatten = nn.Flatten()
        
    def forward(self, x):

        x = self.cnn_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        #print(x.shape)
        
        x = self.cnn_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        #print(x.shape)
        
        x = self.flatten(x)
        #print(x.shape)
        
        x = self.linear_layer_1(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.linear_layer_2(x)
        x = self.dropout(x)
        x = self.relu(x)
        
        x = self.linear_layer_3(x)
        #logits = self.sigmoid(x)
        return x

In [9]:
model = NeuralNetwork(784)
print(model)

NeuralNetwork(
  (cnn_layer_1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn_layer_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_layer_1): Linear(in_features=32768, out_features=900, bias=True)
  (linear_layer_2): Linear(in_features=900, out_features=768, bias=True)
  (linear_layer_3): Linear(in_features=768, out_features=512, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.2, inplace=False)
)


In [10]:
from torchsummary import summary
summary(model,(1,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 128, 128]             416
           Dropout-2         [-1, 16, 128, 128]               0
              ReLU-3         [-1, 16, 128, 128]               0
         MaxPool2d-4           [-1, 16, 64, 64]               0
            Conv2d-5           [-1, 32, 64, 64]          12,832
           Dropout-6           [-1, 32, 64, 64]               0
              ReLU-7           [-1, 32, 64, 64]               0
         MaxPool2d-8           [-1, 32, 32, 32]               0
           Flatten-9                [-1, 32768]               0
           Linear-10                  [-1, 900]      29,492,100
          Dropout-11                  [-1, 900]               0
             ReLU-12                  [-1, 900]               0
           Linear-13                  [-1, 768]         691,968
          Dropout-14                  [

*Optimizer*

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=.001)